# Bzip2

## 0. Préparation : fichier en binaire

En Python on peut facilement ouvrir un fichier en binaire, en précisant `'rb'` comme mode d'ouverture :

In [9]:
with open('../TP4/etranger.txt','rb') as f:
    letters = f.read()

Le résultat `listwords` est de type `bytes` = une séquence d'octets.  Chaque octet est lui-même un entier compris entre 0 et 255.

In [10]:
print(type(letters))
print(type(letters[0]))

<class 'bytes'>
<class 'int'>


Le type `bytes` est itérable, mais si vous préférez travailler avec les listes avec lesquelles vous êtres plus familiers, vous pouvez convertir en liste avec la fonction `list()`

In [11]:
listLetters = list(letters)
print(letters[:5])
print(listLetters[:5])

b'Alber'
[65, 108, 98, 101, 114]


**Important :** dans toutes la suite on travaillera *par défaut* sur l'alphabet contenant tous les octets. Pour tester sur des exemples (ceux du cours ou du TD), il peut néanmoins être utile de mettre les mots sur des alphabets habituels ; on pourra utiliser les fonctions de transformation (on considère que le caractère # utilisé dans BWT est encodé -1 comme ça elle sera directement inférieures à toutes les autres):

In [12]:
def strToOctets(word):
    return [ord(x) if x != '#' else -1 for x in word]

def octetsToStr(listOctets):
    return "".join([chr(x) if x != -1 else '#' for x in listOctets])

word = "ababaac#a"
octets = strToOctets(word)
print(octets)
print(octetsToStr(octets) == word)

[97, 98, 97, 98, 97, 97, 99, -1, 97]
True


## 1. Transformée de Burrows-Wheeler (BWT)

Dans cette première partie, on s'intéresse à la transformée de Burrows-Wheeler d'un mot.

Ecrire la fonction `ordered_suff(u)` qui calcule la table des suffixe d'un mot d'octets `u` qui se termine par `-1`: si `S = ordered_suff(u)` alors
`S` est de longueur `n=len(u)`, contient tous les entiers entre `0` et `len(u)-1` et on a
$$
u[S[0]:\ ] < u[S[1]:\ ] < u[S[2]:\ ] < \ldots < u[S[n-1]:\ ]
$$

Par exemple : si `u=[1,2,2,1,-1]` on a `S=[4,3,0,2,1]` car le plus petit suffixe de `u` est celui qui commence en à l'indice `4`, puis celui commençant à l'indice `3`, etc


In [13]:
import functools

def comp(u,i,j):
    k = 0
    while u[i+k] == u[j+k]:
        k+=1
    return u[i+k] - u[j+k]

def ordered_stuff(u):
    res = list(range(len(u)))
    return sorted(res, key=functools.cmp_to_key(lambda x, y: comp(u, x, y)))

u = [1, 2, 2, 1, -1]
print(ordered_stuff(u))

[4, 3, 0, 2, 1]


**Attention :** il est possible que votre tri soit très lent si vous vous y prenez mal. Si cela prend plus de 5 secondes sur le fichier  `etranger.txt` changez de façon de procéder.

In [14]:
with open('../TP4/etranger.txt','rb') as f:
    letters = list(f.read())

from time import time
letters.append(-1)
t1 = time()
v = ordered_stuff(letters)
print("temps = ", time() - t1)

temps =  1.108100175857544


Comme on a vu en cours, si on a les suffixes dans l'ordre, il suffit de prendre à chaque fois la lettre d'avant pour constituer la transformée de Burrow-Wheeler. Utilisez cette propriété pour encoder `bwt(u)` où `u` est un mot d'octets : on commence par ajouter un `-1` à la fin de `u`, puis on calcule `ordered_suff` sur le résultat, et on en déduit la transformée.

Pour `u = [1,2,0,1,1]` on doit trouver `bwt(u) = [1, 2, 1, 0, -1, 1]`

In [27]:
def bwt(u):
    tmp = list(u) + [-1]
    ordered = ordered_stuff(tmp)
    return [tmp[ordered[i]-1] for i in range(len(ordered))]
    


    
u=[1,2,0,1,1]
print(bwt(u))

[1, 2, 1, 0, -1, 1]


Essayez sur l'étranger avec le script ci-dessous

In [44]:
with open('../TP4/etranger.txt','rb') as f:
    letters = list(f.read())

from time import time
t1 = time()
v = bwt(letters)
print("temps",time() - t1)
print(v[:100])

temps 1.1832551956176758
[10, 10, 10, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 45, 45, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 45, 32, 32, 32, 32, 32, 45, 32, 45, 32, 45, 45, 32, 32, 32, 32, 45, 45, 32, 32, 45, 45, 32, 32, 32, 32, 45, 32, 32, 32, 32, 45, 32, 32, 45, 45, 45, 32, 32, 32, 32, 32, 32, 32, 32, 32]


Encodez la fonction `un_bwt(v)` qui calcule la transformée de BW inverse d'un mot d'octets. On doit avoir `un_bwt(bwt(u)) == u`. Pour cela, on calculera la fonction `phi` comme présenté dans la feuille de TD, puis on l'utilisera comme vu en cours.

In [4]:
def compute_phi(v):
    tmp = dict()
    res = list()
    for elem in v:
        if elem not in tmp.keys():
            tmp[elem] = 1
        else:
            value = tmp.pop(elem)
            res.append((elem, value))
            res.append((elem, value+1))
    res + list(tmp.keys())
    print(res)

def un_bwt(v):
    ...

v = [3,-1,2,2,1,1,1]
compute_phi(v)

[(2, 1), (2, 2), (1, 1), (1, 2)]


Testez avec les scripts suivants

In [29]:
u = [1,0,2,2,1,0,1,1]
print(bwt(u))
print(un_bwt(bwt(u)) == u)

[1, 1, 1, 1, 2, -1, 0, 2, 0]
False


In [30]:
with open('etranger.txt','rb') as f:
    letters = list(f.read())

print(un_bwt(bwt(letters)) == letters)

FileNotFoundError: [Errno 2] No such file or directory: 'etranger.txt'

## 2. Move to Front (MtF)

Pour notre version de Move to Front, on considère que l'alphabet est composé de -1 et des nombres de 0 à 255 :
`A = list(range(-1,256))`. Ecrire la fonction `mtf(v)` qui calcule Move to Front pour une suite de nombres de `A`

In [25]:
def mtf(v):
    A = list(range(-1, 256))
    res = []
    cpt = 0
    while cpt < len(v):
        val = A.index(v[cpt])
        tmp = v[cpt]
        A.remove(tmp)
        res.append(val)
        A.insert(0, tmp)
        cpt+=1
    return res
        

# doit retourner True
print(mtf([1, 1, 1, 1, 2, -1, 0, 2, 0]) == [2, 0, 0, 0, 3, 2, 3, 2, 1])

True


Essayez-le sur l'`etranger.txt`

In [26]:
with open('../TP4/etranger.txt','rb') as f:
    letters = list(f.read())

v = bwt(letters)
from time import time
t1 = time()
w = mtf(v)
print("temps",time()-t1)

temps 0.06488490104675293


Implémentez `un_mtf(w)` qui calcule l'inverse de Move to Front

In [23]:
def un_mtf(w):
    A = list(range(-1, 256))
    cpt = 0
    res = []
    while cpt < len(w):
        tmp = A[w[cpt]]
        res.append(tmp)
        A.remove(tmp)
        A.insert(0, tmp)
        cpt+=1
    return res
    

Le script ci-dessous doit retourner `True`

In [ ]:
print(un_mtf([2, 0, 0, 0, 3, 2, 3, 2, 1]) == [1, 1, 1, 1, 2, -1, 0, 2, 0])

True


### 3. bzip2

Implémentez bzip2 et un_bzip2 en utilisant les fonctions sur Huffman des TPs précédents.

In [22]:
# code pour huffman

def count(L):
   D = {}
   for a in L:
       if a not in D:
           D[a] = 1
       else:
           D[a] +=1
   return [(v,k) for (k,v) in D.items()]

class Heap:
   def __init__(self, function= lambda x:x):
       self.tab = []
       self.f = function
   def father(self,i):
       if i == 0:
           return None
       return int((i-1)/2)
   def left(self,i):
       if 2*i+1 < len(self.tab):
           return 2*i+1
       return None
   def right(self,i):
       if 2*i+2 < len(self.tab):
           return 2*i+2
       return None
   def __str__(self):
       return str(self.tab)
   def add(self,x):
       self.tab.append(x)
       i = len(self.tab)-1
       while i>0 and self.f(self.tab[i]) < self.f(self.tab[self.father(i)]):
           self.tab[i], self.tab[self.father(i)] = self.tab[self.father(i)], self.tab[i]
           i = self.father(i)
   def extract(self):
       if len(self.tab) == 1:
               return self.tab.pop()
       res = self.tab[0]
       self.tab[0] = self.tab.pop()
       i = 0
       while True:
           l,r = self.left(i), self.right(i)
           vl, vr = self.f(self.tab[i]), self.f(self.tab[i])
           if l is not None:
               vl = self.f(self.tab[l])
           if r is not None:
               vr = self.f(self.tab[r])
           if self.f(self.tab[i]) <= vr and self.f(self.tab[i]) <= vl:
               return res
           if vl < vr:
               self.tab[i], self.tab[l] = self.tab[l], self.tab[i]
               i = l
           else:
               self.tab[i], self.tab[r] = self.tab[r], self.tab[i]
               i = r    

def huffmanTree(L):
   C = count(L)
   H = Heap(lambda x:x[0])
   for c in C:
       H.add(c)
   while len(H.tab)>1:
       x1 = H.extract()
       x2 = H.extract()
       A = (x1[0]+x2[0],x1,x2)
       H.add(A)
   return H.tab[0]  

def huffmanToCode(T,C=None,word=None):
   if word is None:
       word = []
   if C is None:
       C = {}
   if len(T) == 2: # leaf
       C[T[1]] = u"".join(word)
   else:
       word.append('0')
       huffmanToCode(T[1],C,word)
       word[-1] = '1'
       huffmanToCode(T[2],C,word)
       word.pop()
   return C

from math import log

def huffmanSize(text):
   K = count(text)
   T = huffmanTree(text)
   C = huffmanToCode(T)
   k = len(K)
   b = int(log(k+1,2)+1)
   return sum([len(C[l])*nb for nb,l in K])+ (b+2)*k -1

In [41]:
def bzip2(u):
    b = bwt(u)
    m = mtf(b)
    return huffmanSize(m)
    
def un_bzip2(v):
    ...

Calculez le taux de compression obtenu avec votre version de bzip2 sur *l'étranger*

In [43]:
with open("../TP4/etranger.txt", "rb") as file:
    letters = list(file.read())
    compression = bzip2(letters)

    print("Taux de compression : ", compression / len(letters) / 8)
    

Taux de compression :  0.31659137131114545
